In [1]:
import tensorflow.keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.utils import to_categorical

import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import random

from tqdm.notebook import tqdm as log_progress

In [2]:
#load and clean data
MODEL_NAME = "Test"
#MODEL_NAME = 'shakespear2.0'
data_file_name = "data.txt"
SEQUENCE_LENGTH = 1 #how many items are fed into the ai per sequence

raw = open("data/" + data_file_name, "r", encoding='utf-8')

lines = []
print("Loading data...")
for line in log_progress(raw):
    if str(line) != '\n':
        lines.append(str(line))

print("Tokenizing...")
#putting the entirety of the input data into one string
text = ""
for line in log_progress(lines):
    text += line
#tokenizer = RegexpTokenizer(r'[a-zA-Z]')
tokenizer = RegexpTokenizer(r'\w+')
#tokenized = tokenizer.tokenize(text)
tokenized = [item for item in text]

#converting the text into numbers to be processed by the embedding layer of the model
words = [] #one of each word in tokenized will be in here
filtering = lambda x : not x in words #for finding if the word should be added to the words array
find = lambda x : float(words.index(x)) if x in words else float(len(words)) #convert each word into a number. -1 means that the item isn't in the vocabulary
normalize = lambda x: [find(i)/len(words) for i in x]

def convert_to_array(word_arr):
    out = []
    for i in range(len(word_arr)):
        arr = [0] * (len(words)+1)
        arr[i] = 1
        out.append(arr)
    return out
    
print("Finding every unique word...")
for word in log_progress(tokenized):
    if filtering(word):
        words.append(word)

#x data is every single word in the data set, in order
#y data is every single word that comes after the corresponding x value
x = []
y = []
print("Compiling dataset...")
for i in log_progress(range(int(len(tokenized) - SEQUENCE_LENGTH))):
    #x.append(convert_to_array(tokenized[i:i+SEQUENCE_LENGTH]))
    
    x.append(normalize(tokenized[i:i+SEQUENCE_LENGTH]))
    y.append(find(tokenized[i+SEQUENCE_LENGTH]))

x = np.asarray(x, np.float32)
y = np.asarray(y, np.float32)

x = np.reshape(x, (x.shape[0], 1, SEQUENCE_LENGTH))
y = to_categorical(y)

print(x.shape)
print(y.shape)

vocab_size = len(words)

Loading data...


0it [00:00, ?it/s]

Tokenizing...


  0%|          | 0/2309 [00:00<?, ?it/s]

Finding every unique word...


  0%|          | 0/99920 [00:00<?, ?it/s]

Compiling dataset...


  0%|          | 0/99919 [00:00<?, ?it/s]

(99919, 1, 1)
(99919, 62)


In [3]:
#create and compile model
model = Sequential()

model.add(LSTM(128, input_shape=(1, SEQUENCE_LENGTH)))
model.add(Dropout(0.3))

model.add(Dense(y.shape[1], activation='softmax'))

opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               66560     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 62)                7998      
                                                                 
Total params: 74,558
Trainable params: 74,558
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
#train model
hist = model.fit(x, y, epochs=200, verbose=1, batch_size=64, shuffle=False, validation_split=0.1)
model.save(MODEL_NAME + ".h5")

plt.title("loss")
plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.show()

plt.title("accuracy")
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.show()

Epoch 1/200
1406/1406 [==============================] - 7s 5ms/step - loss: 2.8986 - accuracy: 0.2144 - val_loss: 2.8903 - val_accuracy: 0.2160
Epoch 2/200
1406/1406 [==============================] - 7s 5ms/step - loss: 2.8959 - accuracy: 0.2145 - val_loss: 2.8875 - val_accuracy: 0.2160
Epoch 3/200
1406/1406 [==============================] - 7s 5ms/step - loss: 2.8948 - accuracy: 0.2138 - val_loss: 2.8853 - val_accuracy: 0.2158
Epoch 4/200
1406/1406 [==============================] - 7s 5ms/step - loss: 2.8925 - accuracy: 0.2136 - val_loss: 2.8833 - val_accuracy: 0.2158
Epoch 5/200
1406/1406 [==============================] - 7s 5ms/step - loss: 2.8902 - accuracy: 0.2140 - val_loss: 2.8814 - val_accuracy: 0.2158
Epoch 6/200
1406/1406 [==============================] - 7s 5ms/step - loss: 2.8888 - accuracy: 0.2139 - val_loss: 2.8801 - val_accuracy: 0.2158
Epoch 7/200
1406/1406 [==============================] - 7s 5ms/step - loss: 2.8873 - accuracy: 0.2142 - val_loss: 2.8789 - val_ac

1406/1406 [==============================] - 7s 5ms/step - loss: 2.7885 - accuracy: 0.2098 - val_loss: 2.7637 - val_accuracy: 0.2187
Epoch 114/200
1406/1406 [==============================] - 7s 5ms/step - loss: 2.7859 - accuracy: 0.2096 - val_loss: 2.7631 - val_accuracy: 0.2187
Epoch 115/200
1406/1406 [==============================] - 7s 5ms/step - loss: 2.7873 - accuracy: 0.2092 - val_loss: 2.7629 - val_accuracy: 0.2187
Epoch 116/200
1406/1406 [==============================] - 7s 5ms/step - loss: 2.7865 - accuracy: 0.2092 - val_loss: 2.7627 - val_accuracy: 0.2187
Epoch 117/200
1406/1406 [==============================] - 7s 5ms/step - loss: 2.7859 - accuracy: 0.2098 - val_loss: 2.7615 - val_accuracy: 0.2188
Epoch 118/200
1406/1406 [==============================] - 7s 5ms/step - loss: 2.7854 - accuracy: 0.2109 - val_loss: 2.7611 - val_accuracy: 0.2187
Epoch 119/200
1406/1406 [==============================] - 7s 5ms/step - loss: 2.7852 - accuracy: 0.2100 - val_loss: 2.7607 - val_ac

KeyboardInterrupt: 

In [33]:
#test model
sample_length = 200
model = load_model("Test.h5")

#get input sentence and process
sentence = input("Enter first " + str(SEQUENCE_LENGTH) + " words...").lower()

root = [item for item in text]
root = normalize(root)
while len(root) < SEQUENCE_LENGTH:
    root.insert(0, 1)

while len(root) > SEQUENCE_LENGTH:
    root.pop(0)

output = sentence + " "
for i in log_progress(range(sample_length)):
    tmp = np.array(root)
    tmp = np.reshape(tmp, (1, 1, SEQUENCE_LENGTH))
    pred = model.predict(tmp, verbose=0)
    next_word = pred.argmax()
    #next_word = np.random.choice(len(pred[0]), p=pred[0])
    output += words[next_word]
    root.pop(0)
    root.append(next_word) #setting the next word

print(output)

Enter first 5 words...hello there



hello there   
  g_"0 ?d ?yzmtw tz
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz
z?
zz

